In [1]:
import fiftyone as fo
from fiftyone import ViewField as F
import pandas as pd
from pathlib import Path

In [2]:
data_df = pd.read_csv('/home/gpubox2/plantin-projects/coins/coin-prepare-meta-dataset/data/preprocessed/raw-dataset-base.csv')

In [3]:
data_df

,coin_id,original_url,coin_side,file_name,name,country,denomination,year,subject,period,coin_type
0,1,https://i.ucoin.net/coin/0/528/528071-1s/polan...,obverse,1.jpg,Poland 1 grosz,Poland,1 grosz,1990-2014,NaN,Third Republic (1990-2022),Circulation coins
1,1,https://i.ucoin.net/coin/0/528/528071-2s/polan...,reverse,2.jpg,Poland 1 grosz,Poland,1 grosz,1990-2014,NaN,Third Republic (1990-2022),Circulation coins
2,1,https://i.ucoin.net/coin/8/947/8947075-1s/pola...,obverse,3.jpg,Poland 1 grosz,Poland,1 grosz,1990-2014,NaN,Third Republic (1990-2022),Circulation coins
3,1,https://i.ucoin.net/coin/8/947/8947075-2s/pola...,reverse,4.jpg,Poland 1 grosz,Poland,1 grosz,1990-2014,NaN,Third Republic (1990-2022),Circulation coins
4,1,https://i.ucoin.net/coin/38/919/38919154-1s/po...,obverse,5.jpg,Poland 1 grosz,Poland,1 grosz,1990-2014,NaN,Third Republic (1990-2022),Circulation coins
...,...,...,...,...,...,...,...,...,...,...,...
525050,80680,https://i.ucoin.net/coin/50/432/50432441-2s/ve...,reverse,525367.jpg,Venice 1 osella,Venice,1 osella,ND (1694),Islands with trees /silver/,Venetian republic (Ducat) (1618-1687),Commemorative coins
525051,80686,https://i.ucoin.net/coin/41/095/41095693-1s/wu...,obverse,525368.jpg,Württemberg 1 ducat,Württemberg,1 ducat,1804,Visit of Friedrich II to the Stuttgart mint,Electorate of Württemberg (1803-1806),Commemorative coins
525052,80686,https://i.ucoin.net/coin/41/095/41095693-2s/wu...,reverse,525369.jpg,Württemberg 1 ducat,Württemberg,1 ducat,1804,Visit of Friedrich II to the Stuttgart mint,Electorate of Württemberg (1803-1806),Commemorative coins
525053,80687,https://i.ucoin.net/coin/41/096/41096186-1s/wu...,obverse,525370.jpg,Württemberg 1 thaler,Württemberg,1 thaler,1795,300th Anniversary-Duchy of Württemberg,Duchy of Württemberg (1737-1802),Commemorative coins


In [45]:
# session = fo.launch_app(dataset='coins-dataset-processed')

In [46]:
dataset = fo.load_dataset('coins-dataset-processed')

uniq_coin_ids = dataset.match(F('country') == 'USA').distinct('coin_id')

data_df.rename(columns={'coin_id': 'internal_coin_id'}, inplace=True)

data_df['coin_id'] = data_df.apply(lambda x: str(x['internal_coin_id'])+'-'+x['coin_side'], axis=1) 

# coin_side = 'obverse'
# and coin_side == @coin_side


group_coin_by_count = data_df.query('internal_coin_id in(@uniq_coin_ids)').groupby('coin_id').size()
selected_coin_ids = group_coin_by_count[group_coin_by_count > 4].index

meta_data = data_df.query('coin_id in(@selected_coin_ids)').groupby('coin_id').first()

In [47]:
group_coin_by_count[group_coin_by_count < 4].shape

(534,)

In [48]:
meta_data.reset_index(inplace=True)

In [49]:
meta_data.shape

(362, 12)

In [50]:
meta_data

,coin_id,internal_coin_id,original_url,coin_side,file_name,name,country,denomination,year,subject,period,coin_type
0,36808-obverse,36808,https://i.ucoin.net/coin/50/796/50796073-1s/us...,obverse,342487.jpg,USA 1 cent,USA,1 cent,2009,200th Anniversary-Birth of Abraham Lincoln-Ear...,United States of America (1981-2023),Commemorative coins
1,36808-reverse,36808,https://i.ucoin.net/coin/50/796/50796073-2s/us...,reverse,342486.jpg,USA 1 cent,USA,1 cent,2009,200th Anniversary-Birth of Abraham Lincoln-Ear...,United States of America (1981-2023),Commemorative coins
2,36809-obverse,36809,https://i.ucoin.net/coin/37/790/37790194-1s/us...,obverse,342510.jpg,USA 1 cent,USA,1 cent,2009,200th Anniversary-Birth of Abraham Lincoln-For...,United States of America (1981-2023),Commemorative coins
3,36809-reverse,36809,https://i.ucoin.net/coin/37/790/37790194-2s/us...,reverse,342511.jpg,USA 1 cent,USA,1 cent,2009,200th Anniversary-Birth of Abraham Lincoln-For...,United States of America (1981-2023),Commemorative coins
4,36810-obverse,36810,https://i.ucoin.net/coin/37/790/37790179-1s/us...,obverse,342534.jpg,USA 1 cent,USA,1 cent,2009,200th Anniversary-Birth of Abraham Lincoln-Pro...,United States of America (1981-2023),Commemorative coins
...,...,...,...,...,...,...,...,...,...,...,...,...
357,732-reverse,732,https://i.ucoin.net/coin/22/582/22582859-2s/us...,reverse,8246.jpg,USA 10 dollars,USA,10 dollars,1866-1907,None,United States of America (1859-1909),Circulation coins
358,733-obverse,733,https://i.ucoin.net/coin/16/533/16533322-1s/us...,obverse,8269.jpg,USA 10 dollars,USA,10 dollars,1907-1908,Indian Head-Eagle,United States of America (1859-1909),Circulation coins
359,733-reverse,733,https://i.ucoin.net/coin/16/533/16533322-2s/us...,reverse,8270.jpg,USA 10 dollars,USA,10 dollars,1907-1908,Indian Head-Eagle,United States of America (1859-1909),Circulation coins
360,79174-obverse,79174,https://i.ucoin.net/coin/50/897/50897371-1s/us...,obverse,517118.jpg,USA ½ dollar,USA,½ dollar,1892-1893,Columbian Exposition,United States of America (1859-1909),Collector coins


In [52]:
meta_data['coin_id'].value_counts()

36808-obverse    1
37209-obverse    1
37208-obverse    1
37207-reverse    1
37207-obverse    1
                ..
36993-reverse    1
36993-obverse    1
36992-reverse    1
36992-obverse    1
79174-reverse    1
Name: coin_id, Length: 362, dtype: int64

In [ ]:
meta_data.to_csv('/home/gpubox2/plantin-projects/coins/similar-search-annotator/api-service/data/USA-coins.csv', index=False)